<a href="https://colab.research.google.com/github/Dan22dongwon/Exploration/blob/main/14_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EB%AA%A8%EB%8D%B8%EC%9D%84_%EC%A0%9C%ED%92%88%EC%9C%BC%EB%A1%9C_%EB%A7%8C%EB%93%A4%EC%96%B4%EB%B3%B4%EC%9E%90_MLOps_%EA%B8%B0%EC%B4%88%5B%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8%5D_MNIST_CNN_%EB%AA%A8%EB%8D%B8_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
import keras_tuner as kt

In [ ]:
# MNIST 데이터셋 불러오기
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
# 데이터 전처리
X_train = x_train.reshape(-1, 28, 28, 1)  # reshaping for convnet
X_test = x_test.reshape(-1, 28, 28, 1)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [ ]:
# CNN 모델 정의
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                                  kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
                                  activation='relu',
                                  input_shape=(28, 28, 1)))
    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=hp.Int('dense_1_units', min_value=128, max_value=512, step=64), activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(10, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Hyperband 튜너 설정
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='mnist')

In [ ]:
# 모델 훈련
tuner.search(X_train, y_train, batch_size=128, epochs=5, validation_data=(X_test, y_test))

1. 하이퍼파라미터 튜닝

In [ ]:
# 최적의 하이퍼파라미터 및 모델 선택
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

In [ ]:
# 최적의 모델 훈련
best_model.fit(X_train, y_train, batch_size=128, epochs=5, validation_data=(X_test, y_test))

In [ ]:
# 테스트 이미지 예측
test_index = np.random.randint(0, X_test.shape[0])
test_image = X_test[test_index]
true_label = np.argmax(y_test[test_index])

prediction = best_model.predict(np.expand_dims(test_image, axis=0))
predicted_label = np.argmax(prediction)

In [ ]:
# 결과 출력
plt.imshow(test_image.reshape(28, 28), cmap='gray')
plt.title(f'True Label: {true_label}\nPredicted Label: {predicted_label}')
plt.axis('off')
plt.show()

2. 모델 배포하기

In [ ]:
# TensorFlow Serving을 설치
!pip install tensorflow-serving-api

3. TFLite 모델 만들기

In [ ]:
# TFLite 모델로 변환
converter = tf.lite.TFLiteConverter.from_keras_model(best_model)
tflite_model = converter.convert()

In [ ]:
# TFLite 모델 저장
with open('best_model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
# TFLite 모델 로드 및 서명 확인
tflite_interpreter = tf.lite.Interpreter(model_content=tflite_model)
tflite_interpreter.allocate_tensors()

In [ ]:
# 모델 정보 출력
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()
print("Input details:", input_details)
print("Output details:", output_details)